# Machine Learning: ***Clasificación.***

## Modelos SVG y KNN.

### Importación de **librerías**:

In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

### Importación y tratamiento del **dataframe**:

In [2]:
# Importación del dataframe:

df_inmueble = pd.read_csv("houses_medellin.csv")
df_inmueble.head()

,Unnamed: 0,Habitaciones,Baños,Estrato,Antigüedad,Piso N°,Administración,Precio m²,Parqueaderos,Estado,Tipo de apartamento,Precio,Área construída (m²),Área privada (m²)
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240000000,NaN,NaN
1,1,1.0,2.0,4.0,1 a 8 años,3,No definida,"$ 4.032.258,06*m²",NaN,NaN,NaN,250000000,62,62
2,2,1.0,2.0,6.0,9 a 15 años,NaN,No definida,"$ 6.428.571,43*m²",2.0,NaN,NaN,675000000,105,0
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,647307563,NaN,NaN
4,4,1.0,1.0,5.0,NaN,3,$ 167.000 COP,$ 4.700.000*m²,2.0,NaN,NaN,235000000,50,0


In [3]:
# Evaluación de la estructura del dataset (tipo de datos):

df_inmueble.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            430 non-null    int64  
 1   Habitaciones          272 non-null    float64
 2   Baños                 271 non-null    float64
 3   Estrato               309 non-null    float64
 4   Antigüedad            217 non-null    object 
 5   Piso N°               214 non-null    object 
 6   Administración        274 non-null    object 
 7   Precio m²             274 non-null    object 
 8   Parqueaderos          212 non-null    float64
 9   Estado                185 non-null    object 
 10  Tipo de apartamento   19 non-null     object 
 11  Precio                430 non-null    int64  
 12  Área construída (m²)  274 non-null    object 
 13  Área privada (m²)     274 non-null    object 
dtypes: float64(4), int64(2), object(8)
memory usage: 47.2+ KB


In [4]:
# La variable "Área construída (m²)" es de tipo "object", pero se requiere que sea de tipo numérico:

df_inmueble["Área construída (m²)"] = df_inmueble["Área construída (m²)"].str.replace(",", ".")
df_inmueble["Área construída (m²)"].astype(float)
df_inmueble.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            430 non-null    int64  
 1   Habitaciones          272 non-null    float64
 2   Baños                 271 non-null    float64
 3   Estrato               309 non-null    float64
 4   Antigüedad            217 non-null    object 
 5   Piso N°               214 non-null    object 
 6   Administración        274 non-null    object 
 7   Precio m²             274 non-null    object 
 8   Parqueaderos          212 non-null    float64
 9   Estado                185 non-null    object 
 10  Tipo de apartamento   19 non-null     object 
 11  Precio                430 non-null    int64  
 12  Área construída (m²)  274 non-null    object 
 13  Área privada (m²)     274 non-null    object 
dtypes: float64(4), int64(2), object(8)
memory usage: 47.2+ KB


In [5]:
# Selección de columnas que se usarán en el análisis:

df_inmueble = df_inmueble[["Habitaciones", "Baños", "Estrato", "Parqueaderos", "Área construída (m²)", "Precio"]]
df_inmueble.head(5)

,Habitaciones,Baños,Estrato,Parqueaderos,Área construída (m²),Precio
0,NaN,NaN,NaN,NaN,NaN,240000000
1,1.0,2.0,4.0,NaN,62,250000000
2,1.0,2.0,6.0,2.0,105,675000000
3,NaN,NaN,NaN,NaN,NaN,647307563
4,1.0,1.0,5.0,2.0,50,235000000


In [6]:
# Eliminación de registros con "null values":

df_inmueble.dropna(inplace=True)
df_inmueble.head(5)

,Habitaciones,Baños,Estrato,Parqueaderos,Área construída (m²),Precio
2,1.0,2.0,6.0,2.0,105,675000000
4,1.0,1.0,5.0,2.0,50,235000000
5,1.0,1.0,4.0,1.0,32,198000000
7,2.0,2.0,6.0,1.0,64,475000000
10,4.0,5.0,6.0,2.0,172,1060000000


In [7]:
# Creación de columna "Categoría":

percentiles = df_inmueble["Precio"].quantile([0.33, 0.67])
rango_bajo = percentiles.iloc[0]
rango_alto = percentiles.iloc[1]

df_inmueble["Categoría"] = pd.cut(df_inmueble["Precio"], bins=[-float("inf"), rango_bajo, rango_alto, float("inf")], labels=["Bajo", "Medio", "Alto"])
df_inmueble.head(5)

,Habitaciones,Baños,Estrato,Parqueaderos,Área construída (m²),Precio,Categoría
2,1.0,2.0,6.0,2.0,105,675000000,Alto
4,1.0,1.0,5.0,2.0,50,235000000,Bajo
5,1.0,1.0,4.0,1.0,32,198000000,Bajo
7,2.0,2.0,6.0,1.0,64,475000000,Medio
10,4.0,5.0,6.0,2.0,172,1060000000,Alto


In [8]:
# Identificación de las categorias a considerar para el análisis (distribución):

df_inmueble["Categoría"].value_counts()

Categoría
Medio    72
Bajo     70
Alto     69
Name: count, dtype: int64

### Creación de **datos de entrenamiento (X, Y)**:

In [9]:
X = df_inmueble.iloc[:, 0:5]
X.head()

,Habitaciones,Baños,Estrato,Parqueaderos,Área construída (m²)
2,1.0,2.0,6.0,2.0,105
4,1.0,1.0,5.0,2.0,50
5,1.0,1.0,4.0,1.0,32
7,2.0,2.0,6.0,1.0,64
10,4.0,5.0,6.0,2.0,172


In [10]:
Y = df_inmueble.iloc[:, 6:7]
Y.head()

,Categoría
2,Alto
4,Bajo
5,Bajo
7,Medio
10,Alto


### Creación del **modelo SVG:**

In [11]:
modelo_svg = SVC()

### Determinación del **"accuracy"** para el modelo SVG:

In [12]:
modelo_svg_score = cross_val_score(modelo_svg, X, Y.values.ravel(), cv=5, scoring="accuracy")
modelo_svg_score

array([0.72093023, 0.78571429, 0.71428571, 0.64285714, 0.78571429])

### Aplicación del modelo SVC mediante **datos de testeo**:

In [13]:
# Creación de datos de testeo (los mismos aplicarán para el modelo KNN):

datos_testeo = pd.DataFrame(
    {
        "Habitaciones": [1.0, 3.0, 6.0],
        "Baños": [1.0, 1.0, 3.0],
        "Estrato": [5.0, 3.0, 5.0],
        "Parqueaderos": [0.0, 1.0, 2.0],
        "Área construída (m²)": [25.0, 40.0, 90.0]
    }
)

datos_testeo

,Habitaciones,Baños,Estrato,Parqueaderos,Área construída (m²)
0,1.0,1.0,5.0,0.0,25.0
1,3.0,1.0,3.0,1.0,40.0
2,6.0,3.0,5.0,2.0,90.0


In [14]:
modelo_svg.fit(X, Y)

c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC()

In [15]:
modelo_svg.predict(datos_testeo)

array(['Bajo', 'Bajo', 'Medio'], dtype=object)

### Creación del **modelo KNN**:

In [16]:
modelo_KNN = KNeighborsClassifier(n_neighbors=3)

### Determinación del **"accuracy"** para el modelo KNN:

In [17]:
modelo_KNN_score = cross_val_score(modelo_KNN, X, Y.values.ravel(), cv=7, scoring="accuracy")
modelo_KNN_score

array([0.83870968, 0.8       , 0.76666667, 0.76666667, 0.73333333,
       0.7       , 0.86666667])

### Aplicación del modelo KNN mediante **datos de testeo**:

In [18]:
modelo_KNN.fit(X, Y)

c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [19]:
modelo_KNN.predict(datos_testeo)

array(['Bajo', 'Bajo', 'Medio'], dtype=object)